##**Custom Keypoint detection using Tensorflow Object detection API**

This notebook helps you train a keypoint detection model on custom dataset annotated in coco format. We are using Centernet-hourglass  model.
This codebase contains scripts for TF record generation, model training and inference.

###Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

###Download Tensorflow model repository into the gdrive workspace

In [ ]:
#and clone the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git

#cd into the TensorFlow directory in your Google Drive
%cd '/content/gdrive/My Drive/TensorFlow'

###Installing dependencies

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

In [ ]:
#reinstall opencv with headless
!pip list | grep opencv
!pip uninstall opencv-python -y
!pip install "opencv-python-headless<4.3"
!pip list | grep opencv

In [ ]:
#cd into 'TensorFlow/models/research'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver .

In [ ]:
# Verify installation is done correctly
#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

###Download pretrained model from TF model garden

In [ ]:
%cd "/content/gdrive/MyDrive/projects/custom_keypoint_detection/pretrained_model"
!wget "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz"
!tar -xvzf "centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz"

###Generate TFrecord from coco annotation dataset

This step has to be executed after the dataset is split for training, validation and testing.

In [ ]:
%cd "/content/gdrive/MyDrive/TensorFlow/models/research/object_detection"
!python dataset_tools/create_coco_tf_record.py --train_image_dir "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/images" \
--test_image_dir "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/images" \
--val_image_dir "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/images" \
--train_annotations_file "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/annotations/train_data.json" \
--testdev_annotations_file "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/annotations/validation_data.json" \
--val_annotations_file "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/annotations/validation_data.json" \
--train_keypoint_annotations_file "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/annotations/train_data.json" \
--val_keypoint_annotations_file "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/annotations/validation_data.json" \
--output_dir "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/tfrecord"

### Run Tensorboard
Open tensorboard to monitor the training performance.

In [ ]:
%cd '/content/gdrive/MyDrive/projects/custom_keypoint_detection/'
#start the Tensorboard
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir=output_models/

### Model Training

In [ ]:
%cd "/content/gdrive/MyDrive/projects/custom_keypoint_detection/"
!python /content/gdrive/MyDrive/TensorFlow/models/research/object_detection/model_main_tf2.py \
--model_dir "output_models" \
--pipeline_config_path "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/pipeline.config"

### Export saved model
Export saved model from the trained checkpoint.

In [ ]:
%cd "/content/gdrive/MyDrive/projects/custom_keypoint_detection/"
!python /content/gdrive/MyDrive/TensorFlow/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path "/content/gdrive/MyDrive/projects/custom_keypoint_detection/dataset/pipeline.config" \
--trained_checkpoint_dir "output_models" \
--output_directory "saved_model"

### Run inference

In [ ]:
%cd "/content/gdrive/MyDrive/projects/custom_keypoint_detection"
!python inference.py